In [1]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
from IPython.display import display

In [2]:
base = importr('base')
df = pd.DataFrame(np.random.random((10, 3)), columns=list('abc'))
r_df = ro.DataFrame({'int_values': ro.IntVector([1,2,3]),
                     'str_values': ro.StrVector(['abc', 'def', 'ghi'])})

## rpy2 doesn't convert pd.DataFrames by default

In [3]:
try:
    base.summary(df)
except NotImplementedError as e:
    print(e)

Conversion 'py2rpy' not defined for objects of type '<class 'pandas.core.frame.DataFrame'>'


## Do not use pd.DataFrame on R DataFrame, the results are transposed and not indexed correctly

In [4]:
pd.DataFrame(r_df)

,0,1,2
0,1,2,3
1,abc,def,ghi


## Convert only on a local scope
https://rpy2.github.io/doc/latest/html/generated_rst/pandas.html

In [5]:
with localconverter(ro.default_converter + pandas2ri.converter):
    display(base.summary(df))
    display(ro.conversion.rpy2py(r_df))
    
try:
    print(base.summary(df))
except NotImplementedError as e:
    print(e)

array(['Min.   :0.007936  ', '1st Qu.:0.181851  ', 'Median :0.371897  ',
       'Mean   :0.427459  ', '3rd Qu.:0.671669  ', 'Max.   :0.951857  ',
       'Min.   :0.1517  ', '1st Qu.:0.4284  ', 'Median :0.4989  ',
       'Mean   :0.4820  ', '3rd Qu.:0.6079  ', 'Max.   :0.6450  ',
       'Min.   :0.03876  ', '1st Qu.:0.15384  ', 'Median :0.37714  ',
       'Mean   :0.42752  ', '3rd Qu.:0.61826  ', 'Max.   :0.95349  '],
      dtype='<U18')

,int_values,str_values
1,1,abc
2,2,def
3,3,ghi


Conversion 'py2rpy' not defined for objects of type '<class 'pandas.core.frame.DataFrame'>'


## Set pandas converter globally

In [6]:
ro.conversion.set_conversion(ro.default_converter + pandas2ri.converter)

In [7]:
base.summary(df)

array(['Min.   :0.007936  ', '1st Qu.:0.181851  ', 'Median :0.371897  ',
       'Mean   :0.427459  ', '3rd Qu.:0.671669  ', 'Max.   :0.951857  ',
       'Min.   :0.1517  ', '1st Qu.:0.4284  ', 'Median :0.4989  ',
       'Mean   :0.4820  ', '3rd Qu.:0.6079  ', 'Max.   :0.6450  ',
       'Min.   :0.03876  ', '1st Qu.:0.15384  ', 'Median :0.37714  ',
       'Mean   :0.42752  ', '3rd Qu.:0.61826  ', 'Max.   :0.95349  '],
      dtype='<U18')

In [8]:
ro.conversion.rpy2py(r_df)

,int_values,str_values
1,1,abc
2,2,def
3,3,ghi


## Return to default converter

In [9]:
ro.conversion.set_conversion(ro.default_converter)

In [10]:
try:
    display(ro.conversion.rpy2py(r_df))
    print(base.summary(df))
except NotImplementedError as e:
    print(e)

int_values,str_values
...,...


Conversion 'py2rpy' not defined for objects of type '<class 'pandas.core.frame.DataFrame'>'
